In [2]:
%load_ext autoreload
%autoreload 2

import fd_imputer
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import itertools

## Set up all paths and labels needed in this notebook

In [85]:
DATA_PATH = 'MLFD_fd_detection/backend/WEB-INF/classes/inputData/adult.csv'
SPLITS_PATH = 'MLFD_fd_detection/data/'
METANOME_DATA_PATH = 'MLFD_fd_detection/backend/WEB-INF/classes/inputData/'
FD_PATH = 'MLFD_fd_detection/results/HyFD-1.2-SNAPSHOT.jar2019-05-07T082200_fds'
DATA_TITLE = 'adult'

## 1. How to split the dataset

In [86]:
df = pd.read_csv(DATA_PATH, sep=';', header=None)
fd_imputer.save_df_split(DATA_TITLE, df, SPLITS_PATH, METANOME_DATA_PATH, [0.8, 0.2])

Dataset successfully written to MLFD_fd_detection/data/adult
Train set successfully written to MLFD_fd_detection/data/train/adult_train
Copied successfully train-dataset to MLFD_fd_detection/backend/WEB-INF/classes/inputData/
Test set successfully written to MLFD_fd_detection/data/test/adult_test


## 2. How to use the ML-imputer

In [87]:
df_train = pd.read_csv(SPLITS_PATH+'test/'+DATA_TITLE+'_test.csv', header=None)
df_test = pd.read_csv(SPLITS_PATH+'train/'+DATA_TITLE+'_train.csv', header=None)
fds = fd_imputer.read_fds(FD_PATH)
impute_column = str(9)
# print(fds)

In [88]:
df_test = df_test.replace('noValueSetHere123156456', np.nan)
df_train = df_train.replace('noValueSetHere123156456', np.nan)

Impute test-set

In [78]:
df_imputed = fd_imputer.ml_imputer(df_train, df_test, impute_column)

2019-05-23 17:40:26,558 [INFO]  CategoricalEncoder for column 9                                found only 63 occurrences of value Amer-Indian-Eskimo
2019-05-23 17:40:26,571 [INFO]  CategoricalEncoder for column 9                                found only 44 occurrences of value Other
2019-05-23 17:40:28,108 [INFO]  
========== start: fit model
2019-05-23 17:40:28,109 [WARNING]  Already bound, ignoring bind()
2019-05-23 17:40:30,164 [INFO]  Epoch[0] Batch [0-184]	Speed: 1460.93 samples/sec	cross-entropy=0.451956	9-accuracy=0.847973
2019-05-23 17:40:32,124 [INFO]  Epoch[0] Train-cross-entropy=0.331838
2019-05-23 17:40:32,125 [INFO]  Epoch[0] Train-9-accuracy=0.885729
2019-05-23 17:40:32,126 [INFO]  Epoch[0] Time cost=4.014
2019-05-23 17:40:32,133 [INFO]  Saved checkpoint to "imputer_model/model-0000.params"
2019-05-23 17:40:32,478 [INFO]  Epoch[0] Validation-cross-entropy=0.157488
2019-05-23 17:40:32,479 [INFO]  Epoch[0] Validation-9-accuracy=0.971037
2019-05-23 17:40:34,559 [INFO]  Epoc

KeyboardInterrupt: 

In [7]:
df_imputed.head()

NameError: name 'df_imputed' is not defined

In [ ]:
f1 = f1_score(df_imputed[impute_column], df_imputed[impute_column+'_imputed'].astype(int), average='weighted')
print(f1)

The ML model's mean relative error is roughly 0.2%

In [ ]:
average_rel_error = ((df_imputed[impute_column] - df_imputed[impute_column+'_imputed'])
 / df_imputed[impute_column]).mean()
print(average_rel_error)

## FD_Imputer

In [84]:
fd = {9: [3, 4, 7, 8]}
lhs = fd[list(fd)[0]]

In [89]:
fd_imputer.fd_imputer(df_test, df_train, fd)

,3,4,7,8,9_imputed,0,1,2,5,6,9,10,11,12,13,14,15
index,,,,,,,,,,,,,,,,,
6463,241998,Bachelors,Prof-specialty,Husband,White,11993,36,Local-gov,13,Married-civ-spouse,White,Male,0,1672,50,United-States,<=50K
23343,475324,Bachelors,Exec-managerial,Husband,White,26938,35,Federal-gov,13,Married-civ-spouse,White,Male,0,0,40,United-States,>50K
19903,149909,HS-grad,Craft-repair,Husband,White,26981,41,Private,9,Married-civ-spouse,White,Male,0,1848,40,United-States,>50K
8795,615367,Bachelors,Sales,Not-in-family,White,27233,23,Private,13,Never-married,White,Female,0,0,40,United-States,<=50K
14438,227310,Some-college,Adm-clerical,Unmarried,Black,20676,46,Federal-gov,10,Never-married,Black,Female,0,0,40,United-States,<=50K
22655,162593,Bachelors,Adm-clerical,Not-in-family,Asian-Pac-Islander,12672,24,Private,13,Never-married,Asian-Pac-Islander,Female,0,0,40,United-States,<=50K
19840,467108,Bachelors,Prof-specialty,Not-in-family,White,12184,30,Private,13,Never-married,White,Male,0,0,45,United-States,<=50K
1758,112137,Some-college,Prof-specialty,Other-relative,Asian-Pac-Islander,22797,21,Private,10,Never-married,Asian-Pac-Islander,Female,0,0,20,South,<=50K
23929,112137,Some-college,Prof-specialty,Other-relative,Asian-Pac-Islander,27046,22,Private,10,Never-married,Asian-Pac-Islander,Female,0,0,20,South,<=50K


In [20]:
pd.merge(df_train.iloc[:,relevant_cols], 
         df_test.iloc[:, :].reset_index(), 
         on=lhs,
        suffixes=('_imputed', '')).set_index('index')

,3,4,7,8,9_imputed,0,1,2,5,6,9,10,11,12,13,14,15
index,,,,,,,,,,,,,,,,,
6463,241998,Bachelors,Prof-specialty,Husband,White,11993,36,Local-gov,13,Married-civ-spouse,White,Male,0,1672,50,United-States,<=50K
23343,475324,Bachelors,Exec-managerial,Husband,White,26938,35,Federal-gov,13,Married-civ-spouse,White,Male,0,0,40,United-States,>50K
19903,149909,HS-grad,Craft-repair,Husband,White,26981,41,Private,9,Married-civ-spouse,White,Male,0,1848,40,United-States,>50K
8795,615367,Bachelors,Sales,Not-in-family,White,27233,23,Private,13,Never-married,White,Female,0,0,40,United-States,<=50K
14438,227310,Some-college,Adm-clerical,Unmarried,Black,20676,46,Federal-gov,10,Never-married,Black,Female,0,0,40,United-States,<=50K
22655,162593,Bachelors,Adm-clerical,Not-in-family,Asian-Pac-Islander,12672,24,Private,13,Never-married,Asian-Pac-Islander,Female,0,0,40,United-States,<=50K
19840,467108,Bachelors,Prof-specialty,Not-in-family,White,12184,30,Private,13,Never-married,White,Male,0,0,45,United-States,<=50K
1758,112137,Some-college,Prof-specialty,Other-relative,Asian-Pac-Islander,22797,21,Private,10,Never-married,Asian-Pac-Islander,Female,0,0,20,South,<=50K
23929,112137,Some-college,Prof-specialty,Other-relative,Asian-Pac-Islander,27046,22,Private,10,Never-married,Asian-Pac-Islander,Female,0,0,20,South,<=50K
